In [0]:
#連結Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#import套件，滿滿的套件
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.optimizers import Adam
from PIL import Image

In [0]:
#讀取train.csv檔案
train = pd.read_csv("gdrive/My Drive/Colab Notebooks/AOI/train.csv")
#宣告變數
train_images = []

#迴圈執行
for i in tqdm(range(train.shape[0])):
  #讀取train_images照片
  img = image.load_img("gdrive/My Drive/Colab Notebooks/AOI/train_images/" + train["ID"][i])
  #將照片進行縮小，縮成(256,256)
  img = img.resize((256,256),Image.LANCZOS)
  #將圖片轉換成數值
  img = image.img_to_array(img)
  #數值正規化
  img = img / 255
  #將數值附加到train_images變數內
  train_images.append(img)

#將數值丟到陣列
X = np.array(train_images)

In [0]:
#讀取train.csv的Label數值
Y = train["Label"].values
#將Label進行onehot
Y = to_categorical(Y)

In [0]:
#分成8:2進行訓練
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 25, test_size = 0.2)
#刪除不必要變數，釋放記憶體
del X, Y, img, train_images, i

In [0]:
#Model開始
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu", input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(6, activation = "sigmoid"))
#編譯Model
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), metrics = ["accuracy"])
#印出Model概況
model.summary()

In [0]:
#訓練Model
model.fit(X_train, Y_train, epochs=200, validation_data=(X_test, Y_test), batch_size = 256)

In [0]:
#讀取test.csv檔案
test = pd.read_csv('gdrive/My Drive/Colab Notebooks/AOI/test.csv')
#宣告變數
test_images = []

#迴圈執行
for i in tqdm(range(test.shape[0])):
  #讀取train_images照片
  img = image.load_img("gdrive/My Drive/Colab Notebooks/AOI/test_images/" + test["ID"][i])
  #將照片進行縮小，縮成(256,256)
  img = img.resize((256,256),Image.LANCZOS)
  #將圖片轉換成數值
  img = image.img_to_array(img)
  #數值正規化
  img = img / 255
  #將數值附加到test_images變數內
  test_images.append(img)

#將數值丟到陣列
test = np.array(test_images)
#預測Label
Label = model.predict_classes(test)

In [0]:
#儲存Label
save['Label'] = Label
#輸出儲存成.csv
save.to_csv('gdrive/My Drive/Colab Notebooks/AOI/Kumu.csv', header=True, index=False)